# Cancer Classification

In [2]:
import jax.numpy as np
import jax
from matplotlib import pyplot as plt
from jax.lib import xla_bridge
import ADAMLL as ada
from sklearn.datasets import load_breast_cancer
from sklearn.model_selection import  train_test_split 
from sklearn.preprocessing import StandardScaler
import seaborn as sns
print("jax backend {}".format(xla_bridge.get_backend().platform))

key = jax.random.PRNGKey(2024)

jax backend gpu


## Data

In [6]:
cancer = load_breast_cancer()
X_train, X_test, y_train, y_test = train_test_split(cancer.data,cancer.target,random_state=0)
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)
X_train = np.asarray(X_train)
X_test = np.asarray(X_test)
y_train = np.asarray(y_train)
y_test = np.asarray(y_test)

### ADAMLL is a local library for machine learning algorithms it contains activation functions, cost functions, and optimization algorithms from previous exercises.

In [39]:
import ADAMLL as ada




#### test and compare with jax to verify the derivations, not used in the analysis

## Running the Neural Network

#### No hidden layer

1
[{'b': Traced<ShapedArray(float32[1])>with<DynamicJaxprTrace(level=1/0)>, 'w': Traced<ShapedArray(float32[2,1])>with<DynamicJaxprTrace(level=1/0)>}]
[Traced<ShapedArray(float32[4,2])>with<DynamicJaxprTrace(level=1/0)>]


TypeError: dot_general requires contracting dimensions to have the same shape, got (1,) and (2,).

### One hidden layer